## LSA (잠재 의미 분석)
- 문서 안에서 단어 사이의 잠재적인 의미 구조를 추출하는 기법
- TF-IDF 방식은 단어 간의 의미적 유사성 반영 X
- TF-IDF에서 SVD 분해를 통해 단어 간의 의미 파악
- TF-IDF에서 차원 축소 (PCA, t-SNE와 같은 축소 기법 사용)하여 관계성 확인
- LSA 효과
    - 벡터 공간의 차원을 줄여서 계산 효율을 증가
    - 비슷한 문맥의 단어를 가까운 벡터로 이동 (의미 유추)</br>예) '영화', '필름'
    - 문서들을 주제별로 분류 (토픽 분석)

- TruncatedSVD (차원 축소 모델)
    - sklearn 안에 위치
    - 절단된 특이값의 분해
    - 고차원 희소 행렬 (값이 0인 행렬)을 낮은 차원으로 압축하여 데이터의 구조적 의미 유지
    - 자연어 처리, 추천 시스템, 의미 분석, 잠재적인 토픽 분석을 주로 사용
    - TF-IDF 행렬은 고차원 -> 저차원
    - 0으로 이루어진 희소행렬들의 구조적인 의미를 유지하면서 값들을 부여할 수 있음
    - 같은 토픽의 문서는 같은 백터 공간에서 가깝게 위치 -> 유사도 기반 자연어 처리에 활용

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from konlpy.tag import Okt

In [2]:
# 토큰화 함수를 정의 -> pos 필터 
okt = Okt()

def tokenize(text):
    result = [ word for word, pos in okt.pos(text) if pos in ['Noun', 'Adjective', 'Verb'] ]
    return result

In [3]:
docs = [
    '이 영화 정말 재미있었다', 
    '매우 연기가 뛰어나다', 
    '이 영화 별로다', 
    '지루한 영화는 보기 어렵다', 
    '정말 훌륭한 연기였다',
    "연기가 별로라서 지루했다"
]

In [4]:
# TF-IDF 벡터화
tfidf = TfidfVectorizer(
    tokenizer=tokenize,
    ngram_range=(1, 1),
    min_df= 1,
    max_df= 0.8,
    sublinear_tf=True,
    lowercase=False
)

In [5]:
X_tfidf = tfidf.fit_transform(docs)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
X_tfidf.shape

(6, 14)

In [7]:
# SVD를 이용한 차원 축소 (LSA 적용)
lsa = TruncatedSVD(n_components=2, random_state=42)
X_lsa = lsa.fit_transform(X_tfidf)

In [8]:
df_lsa = pd.DataFrame(X_lsa, columns = ['topic1', 'topic2'] )
df_lsa

,topic1,topic2
0,0.722615,-0.336336
1,0.229990,0.678032
2,0.801574,-0.279071
3,0.346712,-0.383244
4,0.392081,0.481308
5,0.516723,0.493418


In [9]:
df_lsa['document'] = docs
df_lsa

,topic1,topic2,document
0,0.722615,-0.336336,이 영화 정말 재미있었다
1,0.229990,0.678032,매우 연기가 뛰어나다
2,0.801574,-0.279071,이 영화 별로다
3,0.346712,-0.383244,지루한 영화는 보기 어렵다
4,0.392081,0.481308,정말 훌륭한 연기였다
5,0.516723,0.493418,연기가 별로라서 지루했다


In [10]:
terms = tfidf.get_feature_names_out()
components = lsa.components_

df_terms = pd.DataFrame(components.T, index = terms, 
                        columns = ['topic1', 'topic2'])
df_terms.sort_values('topic1')

,topic1,topic2
뛰어나다,0.083061,0.338339
매우,0.083061,0.338339
보기,0.105700,-0.161434
어렵다,0.105700,-0.161434
지루한,0.105700,-0.161434
였다,0.125589,0.213017
훌륭한,0.125589,0.213017
지루했다,0.200316,0.264294
재미있었다,0.244520,-0.157252
연기,0.283132,0.564684


---

1. ratings_test.txt 파일 로드
2. 결측치제외, id 컬럼 제외
3. document의 중복된 데이터를 제거
4. 상위 5000개를 필터
5. 독립 변수, 종속변수 train, test 로 데이터 분할
6. X_train을 이용하여 tdifd, LSA 작업
7. 모델은 SVC 사용하여 학습 및 평가

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv("../data/ratings_test.txt", sep='\t')
df.dropna(inplace=True)
df.drop('id', axis=1, inplace=True)

In [13]:
# documet 컬럼의 데이터 중 중복 데이터 제거 
df.drop_duplicates('document', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49157 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  49157 non-null  object
 1   label     49157 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [14]:
# 독립, 종속 변수 생성 
X = df['document'].values
Y = df['label'].values

In [15]:
# train, test로 데이터셋 변경
X_tr, X_te, Y_tr, Y_te = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [16]:
# train , test 데이터 개수를 줄여준다( 빠른 실행을 위해 )
# 데이터를 미리 자르면 불균형해질 수 있음
X_tr = X_tr[:5000]
X_te = X_te[:1000]
Y_tr = Y_tr[:5000]
Y_te = Y_te[:1000]

In [17]:
# 토큰화 함수를 생성 
okt = Okt()

def tokenize(text):
    return okt.morphs(text)
# 백터화
vector = TfidfVectorizer(
    tokenizer=tokenize, 
    lowercase= False, 
    ngram_range=(1,2), 
    min_df= 3
)

In [18]:
# train 데이터를 이용하여 백터화 작업 
# X_tr를 이용하여 vector에 학습
vector.fit(X_tr)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,False
,preprocessor,None
,tokenizer,<function tok...0024FE45C0C20>
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [19]:
X_tr_vc = vector.transform(X_tr)
# train 데이터로 학습한 변환 모델에 test데이터로 변환 
X_te_vc = vector.transform(X_te)

In [20]:
svc = SVC(
    kernel='linear', 
    C = 1.0, 
    random_state=42
)

In [21]:
# tf_idf 변환을 한 데이터를 이용하여 SVC 모델에 학습 및 평가 
svc.fit(X_tr_vc, Y_tr)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [22]:
# test데이터를 이용하여 예측
pred_vc = svc.predict(X_te_vc)

In [23]:
# 정확도, f1 score 확인 
acc_vc = accuracy_score(pred_vc, Y_te)
f1_vc = f1_score(pred_vc, Y_te)
print(f"TD-IDF 변환 후 학습 정확도 : {round(acc_vc, 4)} f1 : {round(f1_vc, 4)}")

TD-IDF 변환 후 학습 정확도 : 0.796 f1 : 0.7914


In [24]:
X_tr_vc.shape

(5000, 5807)

In [25]:
# LSA 정의 
lsa = TruncatedSVD(
    n_components= 200, 
    random_state= 42
)
# lsa를 이용하여 학습 
lsa.fit(X_tr_vc)

,n_components,200
,algorithm,'randomized'
,n_iter,5
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,42
,tol,0.0


In [26]:
X_tr_lsa = lsa.transform(X_tr_vc)
X_te_lsa = lsa.transform(X_te_vc) 

In [27]:
svc.fit(X_tr_lsa, Y_tr)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [28]:
pred_svc = svc.predict(X_te_lsa)

In [29]:
acc_lsa = accuracy_score(pred_svc, Y_te)
f1_lsa = f1_score(pred_svc, Y_te)

print(f'LSA 작업 후 정확도 : {round(acc_lsa, 4)}, f1 : {round(f1_lsa, 4)}')

LSA 작업 후 정확도 : 0.744, f1 : 0.7294


In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
std = StandardScaler(with_mean= True)

In [32]:
X_tr_std = std.fit_transform(X_tr_lsa)
X_te_std = std.transform(X_te_lsa)

In [33]:
svc.fit(X_tr_std, Y_tr)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [34]:
pred_std = svc.predict(X_te_std)

In [35]:
acc_std = accuracy_score(pred_std, Y_te)
f1_std = f1_score(pred_std, Y_te)

print(f'TD-IDF -> LSA -> Scaler 작업 후 정확도 : {round(acc_std, 4)} f1 : {round(f1_std, 4)}')

TD-IDF -> LSA -> Scaler 작업 후 정확도 : 0.745 f1 : 0.7341
